# Lab 8


## Setup for SUSY Dataset

Use the SUSY dataset for the rest of this lab. Here is a basic setup.

In [ ]:
# !curl http://archive.ics.uci.edu/ml/machine-learning-databases/00279/SUSY.csv.gz > SUSY.csv.gz

In [58]:
from sh import gunzip
gunzip("SUSYSMALL.csv.gz")

ModuleNotFoundError: No module named 'sh'

In [28]:
# Our usual libraries...
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML, display
import tabulate

In [60]:
# filename = "SUSY-small.csv"
filename="../Lab.7/SUSY.csv"
VarNames=["signal", "l_1_pT", "l_1_eta","l_1_phi", "l_2_pT", "l_2_eta", 
          "l_2_phi", "MET", "MET_phi", "MET_rel", "axial_MET",
          "M_R", "M_TR_2", "R", "MT2", "S_R", "M_Delta_R", "dPhi_r_b", "cos_theta_r1"]
df = pd.read_csv(filename, dtype='float64', names=VarNames)

## Scikit-Learn

[Scikit-learn](http://scikit-learn.org) is a rich python library for data science, including machine learning. For example, we can build a Fisher Discriminant (aka Linear Discriminant Analysis, or LDA). 

### Exercise 1: Install Scikit-Learn

Follow the [Installation Instructions](https://scikit-learn.org/stable/install.html) and install `scikit-learn` in your environment.

### Exercise 2: Read About Classifiers

#### Part a
Scikit-learn offers an impressively comprehensive list of machine learning algorithms. Browse through [scikit-learn's documentation](https://scikit-learn.org/stable/index.html). You'll note the algorithms are organized into classification, regression, clustering, dimensionality reduction, model selection, and preprocessing. Browse through the list of [classification algorithms](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning). 

#### Part b
Note scikit-learn's documentation is rather comprehensive. The documentation on [linear models](https://scikit-learn.org/stable/modules/linear_model.html) shows how classification problems are setup. Read about the first few methods and try to comprehend the example codes. Skim the rest of the document.

#### Part c
Read through the [LDA Documentation](https://scikit-learn.org/stable/modules/lda_qda.html).


### Exercise 3: Training a Classifier

Lets' repeat what we did manually in the previous lab using scikit-learn. We'll use a LDA classifier, which we can instanciate as follows:

In [62]:
import sklearn.discriminant_analysis as DA
Fisher=DA.LinearDiscriminantAnalysis()

As discussed in the lecture, to properly formulate our problem, we'll have to:

* Define the inputs (X) vs outputs (Y)
* Designate training vs testing samples (in order to get a unbias assessment of the performance of Machine Learning algorithms)

for example, here we'll take use 4M events for training and the remainder for testing.

In [64]:
N_Train=400

Train_Sample=df[:N_Train]
Test_Sample=df[N_Train:]

X_Train=Train_Sample[VarNames[1:]]
y_Train=Train_Sample["signal"]

X_Test=Test_Sample[VarNames[1:]]
y_Test=Test_Sample["signal"]

Test_sig=Test_Sample[Test_Sample.signal==1]
Test_bkg=Test_Sample[Test_Sample.signal==0]


We can train the classifier as follow:

In [66]:
Fisher.fit(X_Train,y_Train)

ValueError: Found array with 0 sample(s) (shape=(0, 18)) while a minimum of 2 is required by LinearDiscriminantAnalysis.

We can plot the output, comparing signal and background:

In [68]:
plt.figure()
plt.hist(Fisher.decision_function(Test_sig[VarNames[1:]]),bins=100,histtype="step", color="blue", label="signal",stacked=True)
plt.hist(Fisher.decision_function(Test_bkg[VarNames[1:]]),bins=100,histtype="step", color="red", label="background",stacked=True)
plt.legend(loc='upper right')
plt.show()

ValueError: Found array with 0 sample(s) (shape=(0, 18)) while a minimum of 1 is required by LinearDiscriminantAnalysis.

<Figure size 640x480 with 0 Axes>

#### Part a

Compare ROC curves computed on the test versus training samples, in a single plot. Do you see a bias?

In [ ]:
from sklearn.metrics import roc_curve, auc

train_data = df[:N_Train]
test_data = df[N_Train:]

X_train = train_data[VarNames[1:]]
y_train = train_data["signal"]

X_test = test_data[VarNames[1:]]
y_test = test_data["signal"]

lda_classifier = DA.LinearDiscriminantAnalysis()
lda_classifier.fit(X_train, y_train)

# Predict probabilities for train and test datasets
train_probs = lda_classifier.predict_proba(X_train)[:, 1]
test_probs = lda_classifier.predict_proba(X_test)[:, 1]

# Compute ROC and AUC for train and test sets
fpr_train, tpr_train, _ = roc_curve(y_train, train_probs)
roc_auc_train = auc(fpr_train, tpr_train)

fpr_test, tpr_test, _ = roc_curve(y_test, test_probs)
roc_auc_test = auc(fpr_test, tpr_test)


plt.figure(figsize=(10, 7))
plt.plot(fpr_test, tpr_test, color='blue', lw=2, label=f'Test ROC (AUC = {roc_auc_test:.2f})')
plt.plot(fpr_train, tpr_train, color='black', lw=2, label=f'Train ROC (AUC = {roc_auc_train:.2f})')
plt.plot([0, 1], [0, 1], color='red', lw=1, linestyle='--')  # Diagonal (random guess)

plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curves for Train and Test Samples')
plt.legend(loc="lower right")
plt.show()

#### Part b

Train the Fisher performance of using the raw, features, and raw+features as input. Compare the performance one a single plot. 

In [ ]:
raw_features = ["l_1_pT", "l_1_eta", "l_1_phi", "l_2_pT", "l_2_eta", "l_2_phi", "MET", "MET_phi"]
derived_features = list(set(VarNames[1:]).difference(raw_features))

X_raw = df[raw_features]
X_derived = df[derived_features]
X_all_features = df[VarNames[1:]]
y_all_features = df["signal"]

# Initializing classifiers for each feature set
lda_raw = DA.LinearDiscriminantAnalysis()
lda_derived = DA.LinearDiscriminantAnalysis()
lda_combined = DA.LinearDiscriminantAnalysis()

lda_raw.fit(X_raw, y_all_features)
lda_derived.fit(X_derived, y_all_features)
lda_combined.fit(X_all_features, y_all_features)

# Predicting probabilities for each set
raw_probs = lda_raw.predict_proba(X_raw)[:, 1]
derived_probs = lda_derived.predict_proba(X_derived)[:, 1]
combined_probs = lda_combined.predict_proba(X_all_features)[:, 1]

# Compute ROC and AUC for each set
fpr_raw, tpr_raw, _ = roc_curve(y_all_features, raw_probs)
roc_auc_raw = auc(fpr_raw, tpr_raw)

fpr_derived, tpr_derived, _ = roc_curve(y_all_features, derived_probs)
roc_auc_derived = auc(fpr_derived, tpr_derived)

fpr_combined, tpr_combined, _ = roc_curve(y_all_features, combined_probs)
roc_auc_combined = auc(fpr_combined, tpr_combined)


plt.figure(figsize=(10, 7))
plt.plot(fpr_raw, tpr_raw, color='purple', lw=2, label=f'Raw Features ROC (AUC = {roc_auc_raw:.2f})')
plt.plot(fpr_derived, tpr_derived, color='blue', lw=2, label=f'Derived Features ROC (AUC = {roc_auc_derived:.2f})')
plt.plot(fpr_combined, tpr_combined, color='green', lw=2, label=f'Combined Features ROC (AUC = {roc_auc_combined:.2f})')
plt.plot([0, 1], [0, 1], color='red', lw=1, linestyle='--')  # Diagonal (random guess)

plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curves for Raw, Derived, and Combined Features')
plt.legend(loc="lower right")
plt.show()

### Exercise 4: Comparing Techniques

#### Part a
Select 3 different classifiers from the techniques listed [here](http://scikit-learn.org/stable/supervised_learning.html#supervised-learning) to compare. Note that you can use the multi-layer perceptron to build a deep network, though training may be prohibitively slow. So avoid this technique.

In [ ]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

svm_model = SVC(kernel='linear')
knn_model = KNeighborsClassifier()
dt_model = DecisionTreeClassifier()

# Use a smaller subset of data for quicker execution
!head -500000 SUSY.csv > SUSY-small.csv
small_dataset = pd.read_csv("SUSY-small.csv", dtype='float64', names=VarNames)

# Splitting the smaller dataset: 80% train, 20% test
train_data = small_dataset[100000:]
test_data = small_dataset[:100000]

X_train = train_data[VarNames[1:]]
y_train = train_data["signal"]

X_test = test_data[VarNames[1:]]
y_test = test_data["signal"]

#### Part b

Write a function that takes an instantiated classifier and performs the comparison from part 3b. Use the function on your choice of functions in part a.

In [ ]:
def compare_classifier_performance(classifier, X_raw, X_features, X_combined, y_train):
    classifier.fit(X_raw, y_train)
    raw_importances = classifier.coef_ if hasattr(classifier, 'coef_') else classifier.feature_importances_

    classifier.fit(X_features, y_train)
    feature_importances = classifier.coef_ if hasattr(classifier, 'coef_') else classifier.feature_importances_

    classifier.fit(X_combined, y_train)
    combined_importances = classifier.coef_ if hasattr(classifier, 'coef_') else classifier.feature_importances_

    fisher_scores_raw = np.abs(raw_importances)
    fisher_scores_features = np.abs(feature_importances)
    fisher_scores_combined = np.abs(combined_importances)


    plt.figure(figsize=(10, 6))
    plt.plot(fisher_scores_raw, label='Raw Data', marker='o')
    plt.plot(fisher_scores_features, label='Features', marker='o')
    plt.plot(fisher_scores_combined, label='Raw + Features', marker='o')

    plt.xlabel('Feature Index')
    plt.ylabel('Fisher Score')
    plt.title('Fisher Performance Comparison')
    plt.legend()
    plt.grid(True)
    plt.show()

compare_classifier_performance(svm_model, X_raw, X_features, X_combined, y_train)
compare_classifier_performance(knn_model, X_raw, X_features, X_combined, y_train)
compare_classifier_performance(dt_model, X_raw, X_features, X_combined, y_train)

#### Part c

Use the best method from part c to compute the maximal significance $\sigma_S= \frac{N_S}{\sqrt{N_S+N_B}}$ for the scenarios in lab 5.

In [ ]:
dt_model.fit(X_train, y_train)

predictions = dt_model.predict(X_test)

signal_count = np.sum(predictions == 1)
background_count = np.sum(predictions == 0)

max_significance = signal_count / np.sqrt(signal_count + background_count)

print("Maximal Significance (σ_S):", max_significance

### Exercise 5: Metrics

Scikit-learn provides methods for computing the FPR, TPR, ROC, AUC metrics. For example:

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_Test, Fisher.decision_function(X_Test))

roc_auc = auc(fpr, tpr)

plt.plot(fpr,tpr,color='darkorange',label='ROC curve (area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.show()


#### Part a
TPR/FPR/ROC/AUC are one way of assessing the quality of a classifier. Read about [Precision and Recall](https://en.wikipedia.org/wiki/Precision_and_recall), [Accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision), and [F-score](https://en.wikipedia.org/wiki/F-score).

#### Part b
Look through [model evaluation](https://scikit-learn.org/stable/modules/model_evaluation.html#) documentation. Using scikit-learns tools, compute TPR, FPR, ROC, AUC, Precision, Recall, F1 score, and accuracy for the method you selected in 4c above and each scenario. Make a nice table, which also includes the maximal significance. 


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, \
                            precision_score, recall_score, f1_score, accuracy_score

def compute_classifier_metrics(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    y_prob = classifier.predict_proba(X_test)[:, 1]

    fpr, tpr, _ = roc_curve(y_test, y_prob)

    roc_auc = roc_auc_score(y_test, y_prob)

    precision, recall, _ = precision_recall_curve(y_test, y_prob)

    # Find threshold for maximum F1 score
    best_f1 = 0
    best_threshold = 0
    for threshold in np.unique(y_prob):
        y_pred_thresh = (y_prob >= threshold).astype(int)
        current_f1 = f1_score(y_test, y_pred_thresh)
        if current_f1 > best_f1:
            best_f1 = current_f1
            best_threshold = threshold

    precision_at_best = precision_score(y_test, (y_prob >= best_threshold).astype(int))
    recall_at_best = recall_score(y_test, (y_prob >= best_threshold).astype(int))

    # Accuracy of the classifier
    accuracy = accuracy_score(y_test, y_pred)

    # Maximal significance calculation
    signal_count = np.sum(y_pred == 1)
    background_count = np.sum(y_pred == 0)
    max_significance = signal_count / np.sqrt(signal_count + background_count)

    return {
        'TPR': tpr,
        'FPR': fpr,
        'ROC AUC': roc_auc,
        'Precision': precision_at_best,
        'Recall': recall_at_best,
        'F1 Score': best_f1,
        'Accuracy': accuracy,
        'Max Significance': max_significance
    }

results = {}

metrics = compute_classifier_metrics(dt_model, X_test, y_test)

results_df = pd.DataFrame([metrics])

print(tabulate(results_df, headers='keys', tablefmt='pretty'))